In [ ]:
## This file implements neural networks and logistic regression on p11presabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 98.2% for combination data, and 93.13% for over-sampling data 
## after improvememt and regularization.
## For logistic regression, the accuracy is 98.2% for combination data, and 91.6% for over-sampling data.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 100% for combination data, and 96.95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 97.65% for combination data, and 94.43% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p11presabs_quant.csv')
df.shape

(255, 974)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.971625
1      0.591125
2      0.902750
3      1.169875
4      1.147000
5      1.112875
6      0.936500
7      0.707000
8      0.806375
9      0.921500
10     0.510000
11     0.958000
12     1.036750
13     0.427625
14     1.052250
15     1.079500
16     0.390375
17     0.385125
18     1.025500
19     0.225750
20     0.295875
21     0.348250
22     0.507375
23     1.114000
24     1.089625
25     0.163000
26     0.888125
27     0.198375
28     0.828250
29     1.153375
         ...   
225    0.975300
226    0.698300
227    0.933700
228    1.012100
229    1.104800
230    0.621000
231    0.941250
232    0.928500
233    0.902250
234    0.897000
235    1.047375
236    0.992625
237    0.901250
238    1.075750
239    0.758000
240    0.831750
241    0.891000
242    0.369125
243    0.772000
244    0.986750
245    0.975375
246    1.125250
247    0.828750
248    1.086250
249    0.854750
250    0.905000
251    0.572500
252    1.071000
253    1.000500
254    0.955125
Name: pheno, Length: 255

In [4]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.971625
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.591125
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.902750
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.169875
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.147000


In [5]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [6]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [7]:
df['pheno']
df['pheno'].value_counts()

1    218
0     37
Name: pheno, dtype: int64

In [8]:
df.shape

(255, 974)

In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(255, 973)

In [11]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_10073,group_10075,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [12]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 972) (255,)


In [13]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 201), (1, 166)]


In [14]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218)]


In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [17]:
############# Fully-Connected Neural Network ################

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [19]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [20]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 256 samples, validate on 111 samples
Epoch 1/100
256/256 [==============================] - 0s 471us/step - loss: 0.5709 - accuracy: 0.7227 - val_loss: 0.4510 - val_accuracy: 0.8468
Epoch 2/100
256/256 [==============================] - 0s 96us/step - loss: 0.4521 - accuracy: 0.8281 - val_loss: 0.3931 - val_accuracy: 0.8739
Epoch 3/100
256/256 [==============================] - 0s 119us/step - loss: 0.4179 - accuracy: 0.8438 - val_loss: 0.3677 - val_accuracy: 0.8559
Epoch 4/100
256/256 [==============================] - 0s 112us/step - loss: 0.3890 - accuracy: 0.8477 - val_loss: 0.3309 - val_accuracy: 0.9189
Epoch 5/100
256/256 [==============================] - 0s 109us/step - loss: 0.3504 - accuracy: 0.8906 - val_loss: 0.2997 - val_accuracy: 0.9279
Epoch 6/100
256/256 [==============================] - 0s 119us/step - loss: 0.3400 - accuracy: 0.8672 - val_loss: 0.2770 - val_accuracy: 0.9189
Epoch 7/100
256/256 [==============================] - 0s 118us/step - loss: 0.3206 -

Epoch 57/100
256/256 [==============================] - 0s 114us/step - loss: 0.1429 - accuracy: 0.9570 - val_loss: 0.1162 - val_accuracy: 0.9730
Epoch 58/100
256/256 [==============================] - 0s 105us/step - loss: 0.1426 - accuracy: 0.9336 - val_loss: 0.1149 - val_accuracy: 0.9730
Epoch 59/100
256/256 [==============================] - 0s 97us/step - loss: 0.1454 - accuracy: 0.9414 - val_loss: 0.1138 - val_accuracy: 0.9640
Epoch 60/100
256/256 [==============================] - 0s 96us/step - loss: 0.1375 - accuracy: 0.9531 - val_loss: 0.1145 - val_accuracy: 0.9640
Epoch 61/100
256/256 [==============================] - 0s 111us/step - loss: 0.1387 - accuracy: 0.9609 - val_loss: 0.1216 - val_accuracy: 0.9640
Epoch 62/100
256/256 [==============================] - 0s 109us/step - loss: 0.1420 - accuracy: 0.9531 - val_loss: 0.1121 - val_accuracy: 0.9640
Epoch 63/100
256/256 [==============================] - 0s 106us/step - loss: 0.1385 - accuracy: 0.9570 - val_loss: 0.1122 - v

In [22]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

111/111 [==============================] - 0s 96us/step
combination test accuracy: 97.30%


In [23]:
#### add regularizer and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [24]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 256 samples, validate on 111 samples
Epoch 1/100
256/256 [==============================] - 0s 892us/step - loss: 1.9956 - accuracy: 0.4531 - val_loss: 0.7201 - val_accuracy: 0.6757
Epoch 2/100
256/256 [==============================] - 0s 103us/step - loss: 1.6230 - accuracy: 0.7188 - val_loss: 0.6235 - val_accuracy: 0.8559
Epoch 3/100
256/256 [==============================] - 0s 107us/step - loss: 1.8299 - accuracy: 0.7812 - val_loss: 0.5952 - val_accuracy: 0.8739
Epoch 4/100
256/256 [==============================] - 0s 110us/step - loss: 1.5263 - accuracy: 0.7930 - val_loss: 0.5719 - val_accuracy: 0.8919
Epoch 5/100
256/256 [==============================] - 0s 119us/step - loss: 2.0026 - accuracy: 0.7617 - val_loss: 0.5545 - val_accuracy: 0.8829
Epoch 6/100
256/256 [==============================] - 0s 129us/step - loss: 1.3850 - accuracy: 0.8359 - val_loss: 0.5364 - val_accuracy: 0.9009
Epoch 7/100
256/256 [==============================] - 0s 185us/step - loss: 1.5900 

Epoch 57/100
256/256 [==============================] - 0s 95us/step - loss: 1.7600 - accuracy: 0.8555 - val_loss: 0.3192 - val_accuracy: 0.9459
Epoch 58/100
256/256 [==============================] - 0s 99us/step - loss: 1.6839 - accuracy: 0.8594 - val_loss: 0.3192 - val_accuracy: 0.9459
Epoch 59/100
256/256 [==============================] - 0s 92us/step - loss: 1.7329 - accuracy: 0.8594 - val_loss: 0.3076 - val_accuracy: 0.9459
Epoch 60/100
256/256 [==============================] - 0s 117us/step - loss: 1.5252 - accuracy: 0.8672 - val_loss: 0.3378 - val_accuracy: 0.9550
Epoch 61/100
256/256 [==============================] - 0s 129us/step - loss: 2.2158 - accuracy: 0.8281 - val_loss: 0.3117 - val_accuracy: 0.9459
Epoch 62/100
256/256 [==============================] - 0s 138us/step - loss: 1.7576 - accuracy: 0.8398 - val_loss: 0.3152 - val_accuracy: 0.9459
Epoch 63/100
256/256 [==============================] - 0s 124us/step - loss: 1.6271 - accuracy: 0.8633 - val_loss: 0.3203 - va

In [26]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

111/111 [==============================] - 0s 106us/step
combination test accuracy: 98.20%


In [27]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [28]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [29]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/100
305/305 [==============================] - 0s 350us/step - loss: 0.7150 - accuracy: 0.5115 - val_loss: 0.6421 - val_accuracy: 0.7405
Epoch 2/100
305/305 [==============================] - 0s 96us/step - loss: 0.6205 - accuracy: 0.6984 - val_loss: 0.5470 - val_accuracy: 0.7481
Epoch 3/100
305/305 [==============================] - 0s 109us/step - loss: 0.5655 - accuracy: 0.7410 - val_loss: 0.5427 - val_accuracy: 0.7863
Epoch 4/100
305/305 [==============================] - 0s 108us/step - loss: 0.5309 - accuracy: 0.7803 - val_loss: 0.5078 - val_accuracy: 0.7786
Epoch 5/100
305/305 [==============================] - 0s 109us/step - loss: 0.5007 - accuracy: 0.8000 - val_loss: 0.4566 - val_accuracy: 0.7863
Epoch 6/100
305/305 [==============================] - 0s 107us/step - loss: 0.4751 - accuracy: 0.8098 - val_loss: 0.4504 - val_accuracy: 0.7863
Epoch 7/100
305/305 [==============================] - 0s 104us/step - loss: 0.4618 -

Epoch 57/100
305/305 [==============================] - 0s 112us/step - loss: 0.2623 - accuracy: 0.9016 - val_loss: 0.2840 - val_accuracy: 0.9084
Epoch 58/100
305/305 [==============================] - 0s 103us/step - loss: 0.2733 - accuracy: 0.9082 - val_loss: 0.2938 - val_accuracy: 0.8855
Epoch 59/100
305/305 [==============================] - 0s 99us/step - loss: 0.2741 - accuracy: 0.9016 - val_loss: 0.3808 - val_accuracy: 0.8244
Epoch 60/100
305/305 [==============================] - 0s 99us/step - loss: 0.2575 - accuracy: 0.9115 - val_loss: 0.2811 - val_accuracy: 0.9084
Epoch 61/100
305/305 [==============================] - 0s 101us/step - loss: 0.2653 - accuracy: 0.9016 - val_loss: 0.3399 - val_accuracy: 0.8702
Epoch 62/100
305/305 [==============================] - 0s 114us/step - loss: 0.2711 - accuracy: 0.9016 - val_loss: 0.3855 - val_accuracy: 0.8168
Epoch 63/100
305/305 [==============================] - 0s 157us/step - loss: 0.2550 - accuracy: 0.9115 - val_loss: 0.3088 - v

In [30]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

131/131 [==============================] - 0s 96us/step
over-sampling test accuracy: 90.84%


In [31]:
#### add regularizer and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [32]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/100
305/305 [==============================] - 0s 572us/step - loss: 2.3330 - accuracy: 0.5410 - val_loss: 0.7729 - val_accuracy: 0.7481
Epoch 2/100
305/305 [==============================] - 0s 108us/step - loss: 2.0491 - accuracy: 0.6852 - val_loss: 0.6871 - val_accuracy: 0.7023
Epoch 3/100
305/305 [==============================] - 0s 120us/step - loss: 1.8576 - accuracy: 0.7344 - val_loss: 0.6810 - val_accuracy: 0.6489
Epoch 4/100
305/305 [==============================] - 0s 135us/step - loss: 2.2250 - accuracy: 0.7082 - val_loss: 0.6376 - val_accuracy: 0.8015
Epoch 5/100
305/305 [==============================] - 0s 198us/step - loss: 2.0305 - accuracy: 0.7377 - val_loss: 0.6344 - val_accuracy: 0.7710
Epoch 6/100
305/305 [==============================] - 0s 182us/step - loss: 1.5795 - accuracy: 0.7836 - val_loss: 0.6193 - val_accuracy: 0.8015
Epoch 7/100
305/305 [==============================] - 0s 291us/step - loss: 2.2057 

Epoch 57/100
305/305 [==============================] - 0s 146us/step - loss: 1.4906 - accuracy: 0.8492 - val_loss: 0.4625 - val_accuracy: 0.8779
Epoch 58/100
305/305 [==============================] - 0s 127us/step - loss: 2.0486 - accuracy: 0.7967 - val_loss: 0.4750 - val_accuracy: 0.8702
Epoch 59/100
305/305 [==============================] - 0s 111us/step - loss: 2.2773 - accuracy: 0.8000 - val_loss: 0.4707 - val_accuracy: 0.8779
Epoch 60/100
305/305 [==============================] - 0s 105us/step - loss: 2.5324 - accuracy: 0.7770 - val_loss: 0.4519 - val_accuracy: 0.8779
Epoch 61/100
305/305 [==============================] - 0s 101us/step - loss: 1.9802 - accuracy: 0.8131 - val_loss: 0.4796 - val_accuracy: 0.8779
Epoch 62/100
305/305 [==============================] - 0s 99us/step - loss: 1.5894 - accuracy: 0.8328 - val_loss: 0.4693 - val_accuracy: 0.8779
Epoch 63/100
305/305 [==============================] - 0s 108us/step - loss: 1.4687 - accuracy: 0.8492 - val_loss: 0.4589 - 

In [34]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

131/131 [==============================] - 0s 89us/step
over-sampling test accuracy: 85.50%


In [36]:
#### improve neural network on over-sampling data
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],),activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [37]:
model1_over3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 649us/step - loss: 2.4449 - accuracy: 0.5180 - val_loss: 0.7645 - val_accuracy: 0.5878
Epoch 2/1000
305/305 [==============================] - 0s 122us/step - loss: 2.0854 - accuracy: 0.6230 - val_loss: 0.6843 - val_accuracy: 0.6183
Epoch 3/1000
305/305 [==============================] - 0s 124us/step - loss: 1.9277 - accuracy: 0.6918 - val_loss: 0.6654 - val_accuracy: 0.5878
Epoch 4/1000
305/305 [==============================] - 0s 110us/step - loss: 1.9569 - accuracy: 0.7082 - val_loss: 0.6600 - val_accuracy: 0.5649
Epoch 5/1000
305/305 [==============================] - 0s 117us/step - loss: 2.0649 - accuracy: 0.6787 - val_loss: 0.6498 - val_accuracy: 0.5878
Epoch 6/1000
305/305 [==============================] - 0s 113us/step - loss: 2.0359 - accuracy: 0.6918 - val_loss: 0.6431 - val_accuracy: 0.5954
Epoch 7/1000
305/305 [==============================] - 0s 154us/step - loss: 

305/305 [==============================] - 0s 129us/step - loss: 1.9362 - accuracy: 0.8033 - val_loss: 0.4639 - val_accuracy: 0.8473
Epoch 57/1000
305/305 [==============================] - 0s 130us/step - loss: 1.9526 - accuracy: 0.8230 - val_loss: 0.4362 - val_accuracy: 0.8702
Epoch 58/1000
305/305 [==============================] - 0s 112us/step - loss: 2.3700 - accuracy: 0.7869 - val_loss: 0.4626 - val_accuracy: 0.8473
Epoch 59/1000
305/305 [==============================] - 0s 110us/step - loss: 2.0870 - accuracy: 0.7934 - val_loss: 0.4619 - val_accuracy: 0.8473
Epoch 60/1000
305/305 [==============================] - 0s 131us/step - loss: 1.5757 - accuracy: 0.8230 - val_loss: 0.4087 - val_accuracy: 0.8931
Epoch 61/1000
305/305 [==============================] - 0s 115us/step - loss: 2.0618 - accuracy: 0.7967 - val_loss: 0.4853 - val_accuracy: 0.7863
Epoch 62/1000
305/305 [==============================] - 0s 97us/step - loss: 1.6023 - accuracy: 0.8361 - val_loss: 0.4825 - val_acc

Epoch 112/1000
305/305 [==============================] - 0s 113us/step - loss: 1.9030 - accuracy: 0.8164 - val_loss: 0.4527 - val_accuracy: 0.8855
Epoch 113/1000
305/305 [==============================] - 0s 112us/step - loss: 1.5109 - accuracy: 0.8459 - val_loss: 0.4288 - val_accuracy: 0.9008
Epoch 114/1000
305/305 [==============================] - 0s 112us/step - loss: 2.1471 - accuracy: 0.8033 - val_loss: 0.4528 - val_accuracy: 0.8931
Epoch 115/1000
305/305 [==============================] - 0s 115us/step - loss: 1.4185 - accuracy: 0.8656 - val_loss: 0.4156 - val_accuracy: 0.9008
Epoch 116/1000
305/305 [==============================] - 0s 110us/step - loss: 1.4746 - accuracy: 0.8590 - val_loss: 0.4021 - val_accuracy: 0.9008
Epoch 117/1000
305/305 [==============================] - 0s 102us/step - loss: 2.3359 - accuracy: 0.7902 - val_loss: 0.4353 - val_accuracy: 0.8931
Epoch 118/1000
305/305 [==============================] - 0s 100us/step - loss: 1.8271 - accuracy: 0.8295 - val_

305/305 [==============================] - 0s 93us/step - loss: 2.3417 - accuracy: 0.8131 - val_loss: 0.3728 - val_accuracy: 0.9313
Epoch 168/1000
305/305 [==============================] - 0s 117us/step - loss: 2.3468 - accuracy: 0.8098 - val_loss: 0.3653 - val_accuracy: 0.9313
Epoch 169/1000
305/305 [==============================] - 0s 109us/step - loss: 1.3823 - accuracy: 0.8557 - val_loss: 0.3804 - val_accuracy: 0.9084
Epoch 170/1000
305/305 [==============================] - 0s 97us/step - loss: 1.3680 - accuracy: 0.8590 - val_loss: 0.3901 - val_accuracy: 0.9160
Epoch 171/1000
305/305 [==============================] - 0s 97us/step - loss: 1.8697 - accuracy: 0.8525 - val_loss: 0.3698 - val_accuracy: 0.9237
Epoch 172/1000
305/305 [==============================] - 0s 95us/step - loss: 1.8003 - accuracy: 0.8361 - val_loss: 0.3781 - val_accuracy: 0.9237
Epoch 173/1000
305/305 [==============================] - 0s 97us/step - loss: 2.0151 - accuracy: 0.8230 - val_loss: 0.3905 - val_a

305/305 [==============================] - 0s 100us/step - loss: 2.3134 - accuracy: 0.8066 - val_loss: 0.3718 - val_accuracy: 0.9237
Epoch 223/1000
305/305 [==============================] - 0s 103us/step - loss: 1.7370 - accuracy: 0.8361 - val_loss: 0.3606 - val_accuracy: 0.9313
Epoch 224/1000
305/305 [==============================] - 0s 105us/step - loss: 2.1479 - accuracy: 0.8262 - val_loss: 0.3689 - val_accuracy: 0.9313
Epoch 225/1000
305/305 [==============================] - 0s 93us/step - loss: 1.5657 - accuracy: 0.8525 - val_loss: 0.3747 - val_accuracy: 0.9313
Epoch 226/1000
305/305 [==============================] - 0s 117us/step - loss: 2.0585 - accuracy: 0.8230 - val_loss: 0.3593 - val_accuracy: 0.9313
Epoch 227/1000
305/305 [==============================] - 0s 114us/step - loss: 1.4906 - accuracy: 0.8721 - val_loss: 0.3350 - val_accuracy: 0.9313
Epoch 228/1000
305/305 [==============================] - 0s 126us/step - loss: 1.7796 - accuracy: 0.8361 - val_loss: 0.3392 - v

...

305/305 [==============================] - 0s 98us/step - loss: 2.0395 - accuracy: 0.8328 - val_loss: 0.3734 - val_accuracy: 0.9389
Epoch 830/1000
305/305 [==============================] - 0s 110us/step - loss: 1.3709 - accuracy: 0.8820 - val_loss: 0.3647 - val_accuracy: 0.9389
Epoch 831/1000
305/305 [==============================] - 0s 97us/step - loss: 2.2399 - accuracy: 0.8197 - val_loss: 0.3672 - val_accuracy: 0.9389
Epoch 832/1000
305/305 [==============================] - 0s 90us/step - loss: 1.7310 - accuracy: 0.8557 - val_loss: 0.3513 - val_accuracy: 0.9389
Epoch 833/1000
305/305 [==============================] - 0s 90us/step - loss: 1.7959 - accuracy: 0.8459 - val_loss: 0.3596 - val_accuracy: 0.9389
Epoch 834/1000
305/305 [==============================] - 0s 103us/step - loss: 1.7282 - accuracy: 0.8557 - val_loss: 0.3630 - val_accuracy: 0.9389
Epoch 835/1000
305/305 [==============================] - 0s 97us/step - loss: 1.4650 - accuracy: 0.8787 - val_loss: 0.3614 - val_a

Epoch 885/1000
305/305 [==============================] - 0s 107us/step - loss: 1.6144 - accuracy: 0.8656 - val_loss: 0.3570 - val_accuracy: 0.9389
Epoch 886/1000
305/305 [==============================] - 0s 107us/step - loss: 1.8291 - accuracy: 0.8492 - val_loss: 0.3510 - val_accuracy: 0.9389
Epoch 887/1000
305/305 [==============================] - 0s 108us/step - loss: 1.6356 - accuracy: 0.8590 - val_loss: 0.3769 - val_accuracy: 0.9389
Epoch 888/1000
305/305 [==============================] - 0s 110us/step - loss: 1.6246 - accuracy: 0.8623 - val_loss: 0.3556 - val_accuracy: 0.9389
Epoch 889/1000
305/305 [==============================] - 0s 99us/step - loss: 1.8312 - accuracy: 0.8492 - val_loss: 0.3636 - val_accuracy: 0.9389
Epoch 890/1000
305/305 [==============================] - 0s 92us/step - loss: 1.9471 - accuracy: 0.8361 - val_loss: 0.3473 - val_accuracy: 0.9389
Epoch 891/1000
305/305 [==============================] - 0s 119us/step - loss: 1.6922 - accuracy: 0.8525 - val_lo

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.103460). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


305/305 [==============================] - 0s 117us/step - loss: 1.9207 - accuracy: 0.8459 - val_loss: 0.3728 - val_accuracy: 0.9389
Epoch 936/1000
305/305 [==============================] - 0s 104us/step - loss: 1.6617 - accuracy: 0.8656 - val_loss: 0.3596 - val_accuracy: 0.9389
Epoch 937/1000
305/305 [==============================] - 0s 114us/step - loss: 1.6217 - accuracy: 0.8656 - val_loss: 0.3564 - val_accuracy: 0.9389
Epoch 938/1000
305/305 [==============================] - 0s 93us/step - loss: 1.6285 - accuracy: 0.8623 - val_loss: 0.3575 - val_accuracy: 0.9389
Epoch 939/1000
305/305 [==============================] - 0s 96us/step - loss: 1.7319 - accuracy: 0.8557 - val_loss: 0.3751 - val_accuracy: 0.9389
Epoch 940/1000
305/305 [==============================] - 0s 94us/step - loss: 1.6732 - accuracy: 0.8623 - val_loss: 0.3653 - val_accuracy: 0.9389
Epoch 941/1000
305/305 [==============================] - 0s 97us/step - loss: 2.1727 - accuracy: 0.8295 - val_loss: 0.3726 - val_

305/305 [==============================] - 0s 93us/step - loss: 2.0224 - accuracy: 0.8393 - val_loss: 0.3432 - val_accuracy: 0.9389
Epoch 991/1000
305/305 [==============================] - 0s 95us/step - loss: 1.8009 - accuracy: 0.8590 - val_loss: 0.3582 - val_accuracy: 0.9389
Epoch 992/1000
305/305 [==============================] - 0s 104us/step - loss: 1.8202 - accuracy: 0.8525 - val_loss: 0.3615 - val_accuracy: 0.9389
Epoch 993/1000
305/305 [==============================] - 0s 93us/step - loss: 1.8887 - accuracy: 0.8426 - val_loss: 0.3547 - val_accuracy: 0.9389
Epoch 994/1000
305/305 [==============================] - 0s 96us/step - loss: 1.8239 - accuracy: 0.8459 - val_loss: 0.3490 - val_accuracy: 0.9389
Epoch 995/1000
305/305 [==============================] - 0s 97us/step - loss: 1.8465 - accuracy: 0.8426 - val_loss: 0.3463 - val_accuracy: 0.9389
Epoch 996/1000
305/305 [==============================] - 0s 92us/step - loss: 1.5601 - accuracy: 0.8721 - val_loss: 0.3510 - val_ac

In [39]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

131/131 [==============================] - 0s 118us/step
over-sampling test accuracy: 93.13%


In [ ]:
############ Logistic Regression ############

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [41]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 98.20%


In [43]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 91.60%


In [45]:
############## Random Forest ##############

In [46]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [48]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 96.95%


In [50]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [51]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98076923 0.96078431 0.98039216 1.         0.96078431]
0.9765460030165911


In [52]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.93548387 0.98387097 0.86885246 0.96666667 0.96666667]
0.9443081262118808
